In [1]:
pip install dash plotly pandas geopy scipy dash-leaflet

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dash --upgrade

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install dash-bootstrap-components

Note: you may need to restart the kernel to use updated packages.


In [60]:
from dash import Dash, callback_context
from geopy.distance import geodesic
from scipy.spatial import cKDTree
import numpy as np
import plotly.express as px
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from sklearn.cluster import DBSCAN
from datetime import date
import dash_bootstrap_components as dbc

In [61]:
data = pd.read_parquet("data_mmda_traffic_spatial.parquet")

data['Type'] = (
    data['Type']
    .str.replace("MMDA ALERTL", "", regex=False)
    .str.replace("MMDA ALERT:", "", regex=False)
    .str.strip()
)

data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data['City'] = data['City'].replace({'ParaÃ±aque': 'Parañaque'})
data = data.dropna(subset=['Latitude', 'Longitude'])

aegis_data = pd.read_parquet("AEGISDataset.parquet", columns=['lat', 'lon', 'flood_heig'])
aegis_data = aegis_data.rename(columns={
    'lat': 'Latitude',
    'lon': 'Longitude',
    'flood_heig': 'Flood_Height'
})

aegis_data['Flood_Height'] = pd.to_numeric(
    aegis_data['Flood_Height'], errors='coerce'
).fillna(0).astype(int)

aegis_data = aegis_data[aegis_data['Flood_Height'].between(1, 8)]
aegis_data = aegis_data.dropna(subset=['Latitude', 'Longitude'])

In [62]:
traffic_coords = data[['Latitude', 'Longitude']].drop_duplicates().to_numpy()
flood_coords = aegis_data[['Latitude', 'Longitude']].to_numpy()

traffic_tree = cKDTree(traffic_coords)
radius = 0.15  # 150 meters
indices = traffic_tree.query_ball_point(flood_coords, r=radius)

filtered_aegis_data = aegis_data[np.array([len(idx) > 0 for idx in indices])]

# coords = filtered_aegis_data[['Latitude', 'Longitude']].values
# db = DBSCAN(eps=0.0015, min_samples=2, metric='haversine').fit(np.radians(coords))

# # Add cluster labels
# filtered_aegis_data['Cluster'] = db.labels_

# # Step 3: Group by clusters and select top flood height per cluster
# filtered_aegis_data = (
#     filtered_aegis_data.groupby('Cluster')
#     .apply(lambda x: x.sort_values('Flood_Height', ascending=False).iloc[0])  # Take the highest flood height
#     .reset_index(drop=True)
# )

# # Step 4: Limit to 20 clusters
# filtered_aegis_data = filtered_aegis_data.head(20)

# # Output the results
# print(f"Filtered flood points: {len(filtered_aegis_data)} (original: {len(aegis_data)})")
# print(filtered_aegis_data)


In [63]:
def create_bar_chart_by_city_and_year(filtered_data):
    if filtered_data.empty:
        return "No data available for the selected filters."

    filtered_data['Year'] = pd.to_datetime(filtered_data['Date']).dt.year.astype(str)  # Convert to string for categorical treatment
    aggregated_data = filtered_data.groupby(['City', 'Year']).size().reset_index(name='Accidents')

    bar_chart = px.bar(
        aggregated_data,
        x='City',
        y='Accidents',
        color='Year',
        barmode='group',
        title='Bar Chart: Accidents by City and Year',
        labels={'City': 'City', 'Accidents': 'Number of Accidents', 'Year': 'Year'}
    )

    bar_chart.update_yaxes(
        tickformat='d',
        title="Number of Accidents"
    )

    return bar_chart


def create_incidents_by_city_chart(filtered_data):
    if filtered_data.empty:
        return "No data available for the selected filters."
    filtered_data = filtered_data.dropna(subset=['City'])
    city_counts = (
        filtered_data['City']
        .value_counts()
        .reset_index(name='Count')
        .rename(columns={'index': 'City'})
    )

    bar_chart_incident = px.bar(
        city_counts,
        x='City',
        y='Count',
        title="Bar Chart: Incidents by City",
        labels={'City': 'City', 'Count': 'Number of Incidents'},
        color='Count',
        color_continuous_scale='burgyl'
    )

    bar_chart_incident.update_layout(
        xaxis_title="City",
        yaxis_title="Number of Incidents",
        xaxis_tickangle=-45,
        title_font_size=16,
        title_x=0.5
    )

    return bar_chart_incident


def create_donut_chart(filtered_data):
    if filtered_data.empty:
        return "No data available for the selected filters."

    filtered_data = filtered_data.dropna(subset=['City'])

    city_counts = (
        filtered_data['City']
        .value_counts()
        .reset_index(name='Count')
        .rename(columns={'index': 'City'})
    )

    if city_counts.empty:
        return "No data available for the selected filters."

    donut_chart = px.pie(
        city_counts,
        names='City',
        values='Count',
        title='Donut Chart: Incidents by City',
        hole=0.4
    )
    return donut_chart


def create_monthly_trends_line_chart(filtered_data):
    if filtered_data.empty:
        return "No valid data for creating the line chart."
    filtered_data['Month'] = pd.to_datetime(filtered_data['Date']).dt.to_period('M')
    aggregated_data = filtered_data.groupby('Month').size().reset_index(name='Accidents')
    aggregated_data['Month'] = aggregated_data['Month'].dt.to_timestamp()
    line_chart = px.line(
        aggregated_data,
        x='Month',
        y='Accidents',
        title='Line Chart: Monthly Accident Trends',
        markers=True,
        labels={'Month': 'Month', 'Accidents': 'Number of Accidents'}
    )
    return line_chart



In [64]:
def assign_category(incident_type):
    if isinstance(incident_type, str):
        if incident_type.lower().startswith("vehicular"):
            return "Vehicular Accident"
        elif incident_type.lower().startswith("stalled"):
            return "Stalled"
        elif incident_type.lower().startswith("multiple collision"):
            return "Multiple Collision"
        elif incident_type.lower().startswith("ongoing"):
            return "Ongoing Events"
        else:
            return "Others"
    return "Others"

data['Category'] = data['Type'].apply(assign_category)


In [65]:
app = Dash(external_stylesheets=[dbc.themes.SPACELAB])
# COSMO, SPACELAB

In [66]:
min_date = date(2018, 8, 20)
max_date = date(2020, 12, 27)

app.layout = html.Div(
    style={'backgroundColor': '#485861', 'padding': '20px', 'display': 'flex'},
    children=[
        html.Div(
            style={
                'backgroundColor': '#17232b',
                'padding': '20px',
                'borderRadius': '10px',
                'width': '300px',
                'marginRight': '20px',
                'position': 'fixed',  # filters fixed in one position
                'top': '20px',
                'height': '90vh',
                'overflowY': 'auto'
            },
            children=[
                html.H3("Filters", style={'textAlign': 'center', 'color': '#ffffff'}),
                html.Label("Select City:", style={'color':'#cce0ed'}),
                dcc.Dropdown(
                    id='city-filter',
                    options=[{'label': city, 'value': city} for city in data['City'].dropna().unique()],
                    placeholder="Select a city",
                    multi=True
                ),
                html.Label("Select Date Range:", style={'marginTop': '10px', 'color': '#cce0ed'}),
                dcc.DatePickerRange(
                    id='date-filter',
                    min_date_allowed=min_date,
                    max_date_allowed=max_date,
                    start_date=min_date,  
                    end_date=max_date,
                    display_format='YYYY-MM-DD',
                ),
                html.Label("Select Traffic Direction:", style={'marginTop': '10px', 'color': '#cce0ed'}),
                dcc.Dropdown(
                    id='direction-filter',
                    options=[{'label': dir, 'value': dir} for dir in data['Direction'].dropna().unique()],
                    placeholder="Select direction",
                    multi=True
                ),
                html.Label("Select Flood Height:", style={'marginTop': '10px', 'color': '#cce0ed'}),
                dcc.Dropdown(
                    id='flood-height-filter',
                    options=[
                        {'label': 'Ankle High', 'value': 1},
                        {'label': 'Knee High', 'value': 2},
                        {'label': 'Waist High', 'value': 3},
                        {'label': 'Neck High', 'value': 4},
                        {'label': 'Top of Head High', 'value': 5},
                        {'label': '1-storey High', 'value': 6},
                        {'label': '1.5-storey High', 'value': 7},
                        {'label': '2-storeys or Higher', 'value': 8},
                    ],
                    placeholder="Select flood height",
                    multi=True
                ),
                html.Button(
                    "Apply Filters",
                    id='apply-filters',  # This is where we capture the apply button
                    style={'marginTop': '20px', 'display': 'block', 'width': '100%'},
                    n_clicks=0  # initial value to handle logic
                ),
            ]
        ),
        html.Div(
            style={'flex': 1, 'marginLeft': '320px'},
            children=[
                # Map
                dcc.Graph(id='map', style={'height': '80vh'}),
                html.P("This is a map of traffic incidents and flooding near roads in Manila.", style={'marginBottom': '20px', 'marginTop': '5px',
                                                                                                        'textAlign': 'center', 'color': '#ffffff'}),
                # Row for bar and donut chart
               html.Div(
                    style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '20px'},
                    children=[
                        html.Div(
                            children=[
                              dcc.Graph(id='bar-chart'),
                              html.P("This is a bar chart of the number of traffic incidents by city and year.", style={'marginBottom': '20px', 'marginTop': '5px',
                                                                                                        'textAlign': 'center', 'color': '#ffffff'}),
                            ]
                        ),
                        html.Div(
                            children=[
                              dcc.Graph(id='donut-chart'),
                              html.P("This is a donut chart of the proportion of traffic incidents per city.", style={'marginBottom': '20px', 'marginTop': '5px',
                                                                                                        'textAlign': 'center', 'color': '#ffffff'}),
                            ]
                        )
                    ]
                ),
                # Line chart
                html.Div(
                    style={'marginTop': '20px'},
                    children=[
                        dcc.Graph(id='line-chart', style={'height': '50vh'}),
                        html.P("This is a trendline chart of traffic incidents per month.", style={'marginBottom': '20px', 'marginTop': '5px',
                                                                                                        'textAlign': 'center', 'color': '#ffffff'}),
                    ]
                ),
                # Incidents by City bar chart
                html.Div(
                    style={'marginTop': '20px'},
                    children=[
                        dcc.Graph(id='city-bar-chart', style={'height': '50vh'}),
                        html.P("This is a bar chart of the number of traffic incidents per city.", style={'marginBottom': '20px', 'marginTop': '5px',
                                                                                                        'textAlign': 'center', 'color': '#ffffff'}),
                    ]
                )
            ]
        )
    ]
)


In [67]:
def apply_filters(data, selected_cities, start_date, end_date, selected_directions, selected_flood_heights):
    if not any([selected_cities, start_date, end_date, selected_directions, selected_flood_heights]):
        return data
    
    # Filter by selected cities (if any)
    if selected_cities:
        data = data[data['City'].isin(selected_cities)]
    
    # Filter by date range (start date and end date)
    if start_date:
        data = data[data['Date'] >= pd.to_datetime(start_date)]
    if end_date:
        data = data[data['Date'] <= pd.to_datetime(end_date)]
    
    # Filter by selected directions (if any)
    if selected_directions:
        data = data[data['Direction'].isin(selected_directions)]
    
    # Filter by selected flood heights (if any)
    if selected_flood_heights:
        data = data[data['Flood_Height'].isin(selected_flood_heights)]
    
    return data

In [68]:
# @app.callback(
#     [
#         Output('city-filter', 'value'),
#         Output('date-filter', 'start_date'),
#         Output('date-filter', 'end_date'),
#         Output('direction-filter', 'value'),
#         Output('flood-height-filter', 'value'),
#         Output('reset-filters', 'n_clicks')  # Reset the `n_clicks` counter
#     ],
#     [Input('reset-filters', 'n_clicks')]
# )
# def reset_filters(n_clicks):
#     if n_clicks is None or n_clicks == 0:
#         # If the button hasn't been clicked, don't update anything
#         return dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update

#     # Reset all filters to default values
#     return (
#         None,                   # Reset city dropdown
#         min_date,               # Reset start_date
#         max_date,               # Reset end_date
#         None,                   # Reset direction dropdown
#         None,                   # Reset flood height dropdown
#         0                       # Reset n_clicks to 0 to allow future resets
#     )

In [69]:
# bar chart
@app.callback(
    Output('bar-chart', 'figure'),
    [
        Input('city-filter', 'value'),
        Input('date-filter', 'start_date'),
        Input('date-filter', 'end_date'),
        Input('direction-filter', 'value'),
        Input('flood-height-filter', 'value'),
        Input('apply-filters', 'n_clicks')
    ]
)
def update_bar_chart(selected_cities, start_date, end_date, selected_directions, selected_flood_heights, n_clicks):
    if n_clicks > 0:  # Only filter after the button is clicked
        filtered_data = apply_filters(data, selected_cities, start_date, end_date, selected_directions, selected_flood_heights)
    else:
        # If no filters are applied (i.e., n_clicks is 0), show all data
        filtered_data = data
    
    return create_bar_chart_by_city_and_year(filtered_data)


In [70]:
# donut chart
@app.callback(
    Output('donut-chart', 'figure'),
    [
        Input('city-filter', 'value'),
        Input('date-filter', 'start_date'),
        Input('date-filter', 'end_date'),
        Input('direction-filter', 'value'),
        Input('flood-height-filter', 'value'),
        Input('apply-filters', 'n_clicks')
    ]
)
def update_donut_chart(selected_cities, start_date, end_date, selected_directions, selected_flood_heights, n_clicks):
    if n_clicks > 0:  # Only filter after the button is clicked
        filtered_data = apply_filters(data, selected_cities, start_date, end_date, selected_directions, selected_flood_heights)
    else:
        # If no filters are applied (i.e., n_clicks is 0), show all data
        filtered_data = data
    
    return create_donut_chart(filtered_data)


In [71]:
# line chart
@app.callback(
    Output('line-chart', 'figure'),
    [
        Input('city-filter', 'value'),
        Input('date-filter', 'start_date'),
        Input('date-filter', 'end_date'),
        Input('direction-filter', 'value'),
        Input('flood-height-filter', 'value'),
        Input('apply-filters', 'n_clicks')
    ]
)
def update_line_chart(selected_cities, start_date, end_date, selected_directions, selected_flood_heights, n_clicks):
    if n_clicks > 0:  # Only filter after the button is clicked
        filtered_data = apply_filters(data, selected_cities, start_date, end_date, selected_directions, selected_flood_heights)
    else:
        # If no filters are applied (i.e., n_clicks is 0), show all data
        filtered_data = data
    
    return create_monthly_trends_line_chart(filtered_data)

In [72]:
# bar chart incidents by city
@app.callback(
    Output('city-bar-chart', 'figure'),
    [
        Input('city-filter', 'value'),
        Input('date-filter', 'start_date'),
        Input('date-filter', 'end_date'),
        Input('direction-filter', 'value'),
        Input('flood-height-filter', 'value'),
        Input('apply-filters', 'n_clicks')
    ]
)
def update_city_bar_chart(selected_cities, start_date, end_date, selected_directions, selected_flood_heights, n_clicks):
    if n_clicks > 0:  # Only filter after the button is clicked
        filtered_data = apply_filters(data, selected_cities, start_date, end_date, selected_directions, selected_flood_heights)
    else:
        # If no filters are applied (i.e., n_clicks is 0), show all data
        filtered_data = data
    
    return create_incidents_by_city_chart(filtered_data)

In [73]:
@app.callback(
    Output('map', 'figure'),
    [
        Input('city-filter', 'value'),
        Input('date-filter', 'start_date'),
        Input('date-filter', 'end_date'),
        Input('direction-filter', 'value'),
        Input('flood-height-filter', 'value')
    ]
)
def update_map(selected_cities, start_date, end_date, selected_directions, selected_flood_heights):
    # Default date range
    start_date = start_date or min_date
    end_date = end_date or max_date

    # Filter traffic data
    filtered_data = data.copy()
    if selected_cities:
        filtered_data = filtered_data[filtered_data['City'].isin(selected_cities)]
    if selected_directions:
        filtered_data = filtered_data[filtered_data['Direction'].isin(selected_directions)]
    if start_date:
        filtered_data = filtered_data[filtered_data['Date'] >= pd.to_datetime(start_date)]
    if end_date:
        filtered_data = filtered_data[filtered_data['Date'] <= pd.to_datetime(end_date)]

    # Handle empty traffic data
    if filtered_data.empty:
        traffic_coords = []
    else:
        traffic_coords = filtered_data[['Latitude', 'Longitude']].drop_duplicates().to_numpy()

    # Filter flood data
    flood_filtered = aegis_data.copy()
    if selected_flood_heights:
        flood_filtered = flood_filtered[flood_filtered['Flood_Height'].isin(selected_flood_heights)]

    # Map flood height to color
    def map_flood_height_to_color(flood_height):
        normalized_intensity = (flood_height - 1) / 7  # Normalize flood height [1-8] to [0-1]
        return f"rgba({int(173 + (0 - 173) * normalized_intensity)}, " \
               f"{int(216 + (0 - 216) * normalized_intensity)}, " \
               f"{int(230 + (139 - 230) * normalized_intensity)}, {0.5 + 0.4 * normalized_intensity})"

    flood_filtered['Flood_Color'] = flood_filtered['Flood_Height'].apply(map_flood_height_to_color)

    # Proximity filtering
    if traffic_coords.size > 0:  # Check if traffic_coords is non-empty
        flood_filtered['Is_Near'] = flood_filtered.apply(
            lambda row: any(
                geodesic((row['Latitude'], row['Longitude']), tuple(tc)).meters <= 150 for tc in traffic_coords
            ),
            axis=1
        )
        flood_filtered = flood_filtered[flood_filtered['Is_Near']]

    # Define map center
    if selected_cities and len(selected_cities) == 1:
        center_lat, center_lon = filtered_data.iloc[0]['Latitude'], filtered_data.iloc[0]['Longitude']
    elif selected_cities:
        center_lat, center_lon = filtered_data[['Latitude', 'Longitude']].mean()
    else:
        center_lat, center_lon = 14.5995, 120.9842  # Default to Manila

    # Create traffic incident map
    color_mapping = {
        "Vehicular Accident": "red",
        "Stalled": "yellow",
        "Multiple Collision": "orange",
        "Ongoing Events": "purple",
        "Others": "green",
    }
    map_fig = px.scatter_mapbox(
        filtered_data,
        lat='Latitude',
        lon='Longitude',
        color="Category",
        hover_name="Category",
        hover_data=["Date", "City", "Type", "Location"],
        zoom=10,
        title="Traffic Incidents and Nearby Floods in Manila",
        center={'lat': center_lat, 'lon': center_lon},
        mapbox_style='carto-positron',
        color_discrete_map=color_mapping
    )

    # Add flood points
    for _, row in flood_filtered.iterrows():
        map_fig.add_scattermapbox(
            lat=[row['Latitude']],
            lon=[row['Longitude']],
            mode='markers',
            marker=dict(
                size=25,
                color=row['Flood_Color'],
                opacity=0.5
            ),
            hovertemplate=(
                f"Flood Height: {row['Flood_Height']}<br>"
                f"Latitude: {row['Latitude']}<br>"
                f"Longitude: {row['Longitude']}<br>"
                "<extra></extra>"
            ),
            showlegend=False
        )

    map_fig.update_layout(
        showlegend=True,
        legend=dict(
            traceorder='normal',
            font=dict(size=12),
            orientation='h',
            x=0.5,
            xanchor='center'
        ),
        legend_title="Traffic Incidents"
    )

    return map_fig

In [74]:
if __name__ == "__main__":
    app.run_server(debug=True, port=8053)